In [1]:
import os
import torch
from torch.utils.data import TensorDataset, DataLoader

from config import *

d:\python\python38\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


D:\REPO\github\ML-Notes\Example\CMN-ENG\..\..\Example\CMN-ENG\vocab


In [2]:
"""
读取vocab
"""
en_vocab = torch.load(os.path.join(vocab_path, "en_vocab"))
zh_vocab = torch.load(os.path.join(vocab_path, "zh_vocab"))

In [3]:
"""
读取corpus
"""

sentence = torch.load(os.path.join(corpus_path, 'sentence'))
valid_len = torch.load(os.path.join(corpus_path, 'valid_len'))
en_sentence = sentence[:, 0, :].to(DEVICE)
zh_sentence = sentence[:, 1, :].to(DEVICE)
en_valid_len = valid_len[:, 0, :].to(DEVICE)
zh_valid_len = valid_len[:, 1, :].to(DEVICE)

ds_sentence = TensorDataset(en_sentence, zh_sentence)
dl_sentence = DataLoader(ds_sentence, batch_size=BATCH_SIZE)

ds_valid_len = TensorDataset(en_valid_len, zh_valid_len)
dl_valid_len = DataLoader(ds_valid_len, batch_size=BATCH_SIZE)

print(en_sentence.shape, '-->', zh_sentence.shape)
print(en_valid_len.shape, '-->', zh_valid_len.shape)

torch.Size([28394, 20]) --> torch.Size([28394, 20])
torch.Size([28394, 1]) --> torch.Size([28394, 1])


In [4]:
# 张量指定长度掩蔽，dim=1
def sequence_mask(X: torch.Tensor, valid_len: torch.Tensor, value=0):
    max_len = X.size(1)
    mask = torch.arange((max_len), dtype=torch.float, device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X


X = torch.tensor([[1., 2., 7.], [3., 4., 8.]])
valid_len = torch.tensor([1, 2])
print(valid_len[:, None])
sequence_mask(X, torch.tensor([1, 2]))

tensor([[1],
        [2]])


tensor([[1., 0., 0.],
        [3., 4., 0.]])

In [5]:
"""
掩蔽交叉熵损失
"""
from torch import nn


class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """
    pred:(batch_size, seq_len, vocab_size)
    label:(batch_size, seq_len)
    valid_len:(batch_size, )

    """

    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction = 'none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(
            pred.permute(0, 2, 1), label)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss


loss = MaskedSoftmaxCELoss()
l = loss(torch.ones(3, 4, 100), torch.ones((3, 4), dtype=torch.long),
         torch.tensor([4, 2, 0]))
print('loss:', l)

loss: tensor([4.6052, 2.3026, 0.0000])


In [6]:
from NeuralNetwork import Seq2Seq

model = Seq2Seq.EncoderDecoder().to(DEVICE)
loss_func = MaskedSoftmaxCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


def train_step(model, features, labels, features_valid_len, labels_valid_len):
    # 正向传播求损失
    predictions = model.forward(features)
    loss = loss_func(predictions, labels, labels_valid_len)
    # 反向传播求梯度
    loss.sum().backward()
    # 参数更新
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()


TypeError: __init__() missing 2 required positional arguments: 'encoder' and 'decoder'